### Odwiedzi  nas czy wystawi? 
 
> **1. Dane:**


_Nasze dane opisują informacje dotyczące rezerwacji w hotelach w Portugalii takie jak:_


* daty rezerwacji, 
* długosc pobytu, 
* ilość gości wraz z ilościa dzieci oraz niemowląt,
* rezerwacje w podziale na segment rynku,
* liczba zarezerwowanych miejsc parkingowych
oraz wiele innych.


_Dane personalne nie występują w tym zbiorze danych i obejmują okolo 120 tysięcy rekordów.
Dane zostały pobrane z [Kaggle](https://www.kaggle.com/datasets/jessemostipak/hotel-booking-demand)_


> **2. Cel prezentacji:**


* Prognozowanie czy rezerwacja zostanie wykorzystana  czy anulowana 
* Aplikacja do sprawdzania zarezerwowoanych pobytów
* Propozycja dla biznesu: jakie korzyści finansowe da wdrożenie wybranego przez nas modelu i aplikacji.

### Analiza danych


![bar1](pictures/13.png)
* Wykres przedstawiający średnią ilość rezerwacji odwołanych i nieodwołanych w miesiącu. Największą ilość rezerwacji odnotowano w miesiącach wakacyjnych, a najmniejszą w zimowych. W każdym miesiącu ilośc rezerwacji odwołanych jest większa niż nieodwołanych.

![linear1](pictures/2.png)
* Wykres przedstawiający średnią cenę za pokój w miesiącu. Największą cenę odnotowano w miesiącach wakacyjnych, a najmniejszą w zimowych.

![bar2](pictures/3.png)
* Wykres przedstawia ilość nowych gości i gości, którzy już wczesniej na odwiedzili z podziałem na rezerwacje odwołanie i nieodwołane. Wśród gość odwiedzających hotel poraz kolejny zdecydowana wiąkszośc nie odwołuje rezerwacji.



In [22]:

import pandas as pd
import plotly.express as px
import warnings
data = {"is_canceled": [0,0,0,1,1,1],
        "deposit_type":['Brak zaliczki', 'Zaliczka bezzwrotna', 'Zaliczka zwrotna', 'Brak zaliczki', 'Zaliczka bezzwrotna', 'Zaliczka zwrotna'],
        "count": [74947, 93, 126, 29694, 14494, 36]}
data = pd.DataFrame(data)
data

px.bar(data, x = "is_canceled", y = "count", color='deposit_type', barmode='group', labels={
                     "is_canceled": "Status",
                     "count": "Ilość rezerwacji",
                     'deposit_type': "Rodzaj zaliczki"
                 },title="Iolość rezerwacji w zależności od rodzaju zaliczki")

* Najwięcej jest rezerwacji w których nie pobrano zaliczki
* Wśród gośc z bezzwrotną zaliczką, większość odwołała rezerwacjię, co jest przeciwne do tego czego można się spodziewać
* Odsenek gości z zaliczką zwrotną jest niewielki

![Wykres korelacji](pictures/korelacja2.png)

### Wybór najlepszego modelu

|**Nazwa modelu**           |**Accuracy score** |**F-beta score**| **Time**    |
| ------------------------- |:-----------------:| --------------:|-------------|
| Dummy_model               |       0.5337      |    0.3686      |   0.1801    |
| DecisionTree_model        |       0.8278      |    0.7653      |   4.4953    |
| **RandomForest_model**    |     **0.8806**    | **0.8655**     |**83.6543**  | 
| LogisticRegression_model  |       0.7805      |    0.7258      |   9.8443    |
| XGBoost_model             |       0.8294      |    0.8027      |  18.3464    | 
| Gaussian_model            |       0.5331      |    0.4917      |   0.7078    |
| KNN_model                 |       0.7643      |    0.6855      |  21.0165    |

* ##### Przetestowaliśmy siedem różnych modeli. Po otrzymaniu wyników accuracy i F-bety zastanawialiśmy się nad modelami: Random Forest oraz XGBoost. 
* ##### Model XGBoost uzyskał niewiele niższe wyniki accuracy i F-bety, za to czasowo okazał się korzystniejszy. 
* ##### Mimo tego, że model Random Forest jest najbardziej "czasochłonny", uznaliśmy, że jest najbardziej miarodajny i idealny dla naszego biznesu.

### Etapy trenowania modelu

 !["trenowanie rf"](pictures/7.png)        !["trenowanie xgb"](pictures/8.png)  

### Wizualizacja wyników

* Krzywa ROC

![Krzywa ROC](pictures/roc2.png)

* Powyższy wykres ilustruje krzywe ROC (oraz wyniki AUC) dla każdego testowanego modelu. 
* Krzywe te idealnie pokazują znikome różnice między modelami Random Forest i XGBoost, na korzyść modelu, który wybraliśmy.
* Różnica w miarze AUC między tymi wynikami jest tak mała, że zaokrąglenie do liczb setnych, nie uwzględnia nawet tych różnic.

* ### Macierz pomyłek

* ##### Macierz pomyłek dobrze porównuje liczbę błędnie i poprawnie oszacowanych i prawdziwych wartości.
* ##### Porównajmy jak wyglądają macierze dla naszych dwóch modeli: Random Forest i XGBoosta.

 !["matrix rf"](pictures/9.png)        !["matrix xgb"](pictures/10.png)  

### Najważniejsze zmienne dla modelu Random Forest 

* ##### Użyliśmy funkcji feature_importances_, która określiła nam "ważność" poszczególnych zmiennych. 

![feature importances](pictures/11.png)

* ##### Z otrzymanych informacji wybraliśmy zmienne o wartości istotności powyżej 0.2.
* ##### Zmienne te wykorzystaliśmy do optymalizacji wybranego przez nas modelu.

### Optymalizacja modelu **Random Forest**

|**Nazwa modelu**           |**Accuracy score** |**F-beta score**| **Time**    |
| ------------------------- |:-----------------:| --------------:|-------------|
| RandomForest_model        |       0.8809      |    0.8649      |   81.6529   |

* ##### Porównanie

**Przed optymalizacją**

|**Nazwa modelu**           |**Accuracy score** |**F-beta score**| **Time**    |
| ------------------------- |:-----------------:| --------------:|-------------|
| RandomForest_model        |       0.8806      |   0.8655       |  83.6543    | 

**Po optymalizacji**

|**Nazwa modelu**           |**Accuracy score** |**F-beta score**| **Time**    |
| ------------------------- |:-----------------:| --------------:|-------------|
| RandomForest_model        |       0.8809      |    0.8649      |   81.6529   |

Użyte przez nas parametry:
* max_depth=None, 
* min_samples_leaf=2, 
* n_jobs=1, 
* random_state=100, 
* max_features='sqrt', 
* n_estimators=200

oraz zmienne, które zostały uznane za najbardziej istotne, nie poprawiły, wyników naszego modelu i nieznacznie poprawiły czas.

* Optymalizacja okazała się zbędna- prawdopodobie jest to spowodowane tym, że zmienne wykazują korelacje na bardzo niskim poziomie.

### Jakie korzyści może przynieść dla biznesu użycie naszego modelu?

* cena za pokój jest stała i jest to średnia cena za pokój w danym miesiącu
* symulacja została przeprowadzona na zbiorze testowym 
* uzyte wartości y są równe wartościom predict uzyskanym za pomocom modelu

In [21]:


warnings.filterwarnings("ignore")
data_df = pd.read_csv("symulacja_4.csv")
data_df
for el in range(len(data_df['status'])):
    if data_df['status'][el] == "overall":
        data_df['status'][el] = "Całość"
    elif data_df['status'][el] == "non calceled + non refund deposit":
        data_df['status'][el] = "Pewny Przychód"
    elif data_df['status'][el] == "non calceled + non refund deposit + extra deposit":
        data_df['status'][el] = "Przychód z zaliczką"
    elif data_df['status'][el] == "non calceled + non refund deposit + extra deposit + 25% discount":
        data_df['status'][el] = "Obniżka 1."
    elif data_df['status'][el] == "non calceled + non refund deposit + extra deposit + 25% discount + 50% discount":
        data_df['status'][el] = "Obniżka 2."
data_df

px.bar(data_df, x = 'month', y = 'cena_calkowita', color='status', barmode='group', labels={
                     'month': "Miesiąc",
                     'cena_calkowita': "Przychód",
                     'status': 'Status'
                 },title="Średni przychód na miesiąc")

* niebieski słupek przedstawia ile hotel zarobiłby, gdyby żadna rezerwacja nie została odwołana,
* czerwony- wszystkie rezerwacje przewidzianeze statusem odwołane zostały odwołane oraz pobrano bezzwrotną zaliczkę 30%
* zielony- gdyby od wszystkich rezerwacji odwołanych została pobrana bezzwrotna zaliczka 30%
#### Wszytkie rezerwacje odwołane mają szanse być ponownie zarezerwowane
* jezeli zostały odwiłane najpóźniej na 14 dni przed planowanym przybyciem gościa, można zaproponawąc potencjalnym klienton 25% zniżki na zwolnione pokoje,
  fioletowy słupek przedstawia ile moglibyśmy zarobić, gdyby 50% pokoi zostało zarezerwowane ponownie z 25% zniżką
* jezeli zostały odwiłane później niż 14 dni przed planowanym przybyciem gościa, można zaproponawąc potencjalnym klienton 50% zniżki na   zwolnione pokoje,
  pomarańczowy słupek przedstawia ile moglibyśmy zarobić, gdyby 50% pokoi zostało zarezerwowane ponownie z 50% zniżką, Takich rezerwacji jest niewiele,
  dlatego róznica między pomarańczowym a fioletowym słupkiem jest niewielka.

Wprowadzając bezzwrotną zaliczkę do wszystkich rezerwacji, nawet jeżli wszytkie rezerwacje, które przewidzi model zostaną odwołane, pewny jest uzysk wysokości zielonego słupka. Obie promocje zostały zasymulowane zakładając, że 50% zwolnionych pokoi zostanie ponownie zarezerwowane, ale należy brać pod uwagę, żę nikt nie skorzysta z promocji. Warto zauważyć, że gdyby wszystkie pokoje zostałt zarezerwowane, uzysk byłby większy niż gdyby żaden pokój nie został odwołany. Analizując obłożenie hotelu, które w miesiącach wakacyjnych jest największe możemy zakładać, że sporo zwolnionych miejsc zostanie ponownie zarezerwowane.

### Aplikacja wykorzystująca model